# Première implementation des modèles

Ce notebook est fait pour faire des tests sur l'implémentation des modèles, de la génération des données, des graphiques, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nprd
import pandas as pd

from smc_movement_models.plots import plot_graph_values, plot_real_data

In [ ]:
gv_fig = plot_graph_values(figsize=None)

In [ ]:
day_fig = plot_real_data("data/19_01_2008.csv", figsize=(10, 5))

In [ ]:
day = pd.read_csv("data/19_01_2008.csv", parse_dates=[0])

In [ ]:
import particles
import particles.distributions as dists
import particles.state_space_models as ssm


class MarineSSM(ssm.StateSpaceModel):
    def PX0(self):  # Distribution of X_0
        return dists.Normal()  # X_0 ~ N(0, 1)

    def PX(self, t, xp):  # Distribution of X_t given X_{t-1}
        return dists.Normal(loc=xp)  # X_t ~ N( X_{t-1}, 1)

    def PY(self, t, xp, x):  # Distribution of Y_t given X_t (and X_{t-1})
        return dists.Normal(loc=x, scale=self.sigma)  # Y_t ~ N(X_t, sigma^2)

In [ ]:
my_model = ToySSM(sigma=0.2)
x, y = my_model.simulate(200)  # sample size is 200

In [ ]:
alg = particles.SMC(fk=ssm.Bootstrap(ssm=my_model, data=y), N=200)
alg.run()